## Evaluation Plots

In [ ]:
import warnings
warnings.filterwarnings("ignore", "Wswiglal-redir-stdio")

In [ ]:
import os
import h5py
import math
import json
import pickle
import corner
import scipy
import itertools
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import sklearn.metrics as metrics
import matplotlib.lines as mlines

import matplotlib as mpl
from functools import partial
from matplotlib.ticker import AutoMinorLocator
from matplotlib.colors import LinearSegmentedColormap, ListedColormap
from matplotlib.cm import ScalarMappable
from matplotlib.colors import Normalize
import matplotlib.patches as mpatches
import matplotlib.colors as mcolors

In [ ]:
import sys
sys.path.insert(0, "../")
from evaluator import *

In [ ]:
# Access results file for D4 testing
# parent_dir = "/local/scratch/igr/nnarenraju/testing_month_D4_seeded/"
parent_dir = "/home/nnarenraju/Research/ORChiD/test_data_d4/"
foreground = os.path.join(parent_dir, 'foreground.hdf')
background = os.path.join(parent_dir, 'background.hdf')
## BEST June
foutput = os.path.join(parent_dir, 'testing_foutput_BEST_June.hdf')
boutput = os.path.join(parent_dir, 'testing_boutput_BEST_June.hdf')
## BEST Feb
# foutput = os.path.join(parent_dir, 'testing_foutput_BEST_Feb.hdf')
# boutput = os.path.join(parent_dir, 'testing_boutput_BEST_Feb.hdf')
## Kennebec Annealed
# foutput = os.path.join(parent_dir, 'testing_foutput_annealed_training.hdf')
# boutput = os.path.join(parent_dir, 'testing_boutput_annealed_training.hdf')
## Metric latest?
# foutput = os.path.join(parent_dir, 'testing_foutput_metric_latest.hdf')
# boutput = os.path.join(parent_dir, 'testing_boutput_metric_latest.hdf')
## D3 Sage
# foutput = os.path.join(parent_dir, 'testing_foutput_D3_SageNet.hdf')
# boutput = os.path.join(parent_dir, 'testing_boutput_D3_SageNet.hdf')
## uTau noPE
# foutput = "/local/scratch/igr/nnarenraju/testing_month_D4_seeded/uTau_unbounded_noPE/testing_foutput.hdf"
# boutput = "/local/scratch/igr/nnarenraju/testing_month_D4_seeded/uTau_unbounded_noPE/testing_boutput.hdf"

injections = os.path.join(parent_dir, 'injections.hdf')

In [ ]:
far_scaling_factor = 2592000.0 # 30 days in seconds

### Preparation steps

In [ ]:
# Find indices contained in foreground
print("Finding injections contained in data")
padding_start, padding_end = 30, 30
dur, idxs = find_injection_times([foreground],
                                 injections,
                                 padding_start=padding_start,
                                 padding_end=padding_end)
if np.sum(idxs) == 0:
    msg = 'The foreground data contains no injections! '
    msg += 'Probably a too small section of data was generated. '
    msg += 'Please make sure to generate at least {} seconds of data. '
    msg += 'Otherwise a sensitive distance cannot be calculated.'
    msg = msg.format(padding_start + padding_end + 24)
    raise RuntimeError(msg)

In [ ]:
# Get injections optimal SNRs
_parent_dir = "/local/scratch/igr/nnarenraju/testing_month_D4_seeded/"
snrs_path = os.path.join(_parent_dir, "snr.hdf")
if os.path.exists(snrs_path):
    with h5py.File(snrs_path, 'r') as fp:
        snrs = fp['snr'][()]

snrs = np.array(snrs)
snrs = snrs[idxs]

In [ ]:
# This checks out
print('Duration calculated by find_injection_times = {}'.format(dur))

### Read Results

In [ ]:
dataset = 4

team_1 = {'name': 'Sage'}
team_2 = {'name': 'PyCBC'}

In [ ]:
injparams = {}
with h5py.File(injections, 'r') as fp:
    params = list(fp.keys())
    for param in params:
        data = fp[param][()]
        injparams[param] = data[idxs]
        
    use_chirp_distance = 'chirp_distance' in params

other_results = "/home/nnarenraju/Research/ORChiD/results"
other_teams = os.listdir(other_results)

print('Dataset {} comparing {} against {}'.format(dataset, team_1['name'], team_2['name']))
team_1['fgpath'] = foutput
team_1['bgpath'] = boutput
    
if team_2['name'] == 'PyCBC':
    team_2['fgpath'] = os.path.join(other_results, "{}/ds{}/fg.hdf".format(team_2['name'], dataset))
    team_2['bgpath'] = os.path.join(other_results, "{}/ds{}/bg.hdf".format(team_2['name'], dataset))
if team_2['name'] == 'aresgw':
    team_2['fgpath'] = "/home/nnarenraju/Research/ORChiD/gw-detection-deep-learning/results_best/fg.hdf"
    team_2['bgpath'] = "/home/nnarenraju/Research/ORChiD/gw-detection-deep-learning/results_best/bg.hdf"


for nteam in [1, 2]:
    team = locals()["team_{}".format(nteam)]
    print('\nTeam {}'.format(team))
    # Read foreground events
    print(f'Reading foreground events from {team["fgpath"]}')
    fg_events = []
    with h5py.File(team['fgpath'], 'r') as fp:
        fg_events.append(np.vstack([fp['time'], fp['stat'], np.array(fp['var'])]))
    team['fgevents'] = np.concatenate(fg_events, axis=-1)
    
    # Read background events
    print(f'Reading background events from {team["bgpath"]}')
    bg_events = []
    with h5py.File(team['bgpath'], 'r') as fp:
        bg_events.append(np.vstack([fp['time'], fp['stat'], np.array(fp['var'])]))
    team['bgevents'] = np.concatenate(bg_events, axis=-1)

### Evaluating Results

In [ ]:
""" Calculate the false-alarm rate and sensitivity of a search algorithm. """

# Get data from fg and bg events file
print('Team 1: {}'.format(team_1['name']))
print('Team 2: {}'.format(team_2['name']))

In [ ]:
# Add SNRs into the injparams (this will automagically include it within most plots)
chirp_distance = use_chirp_distance
injparams['snr'] = snrs
output_dir = "./evaluation_plots/"
# Return data tmp var
ret = {}

## COMMON ##
# Get injection params
injtimes = injparams['tc']
dist = injparams['distance']

# Get chirp mass from the source masses
if chirp_distance:
    massc = mchirp(injparams['mass1'], injparams['mass2'])
# Set duration if nothing is passed
duration = dur

In [ ]:
def network_output(found_injections, noise_stats, output_dir, team_name, lower_threshold=0.0):
    # Plotting the noise and signals stats for found samples
    plt.figure(figsize=(12.0, 12.0))
    foo = found_injections[1][found_injections[1] > lower_threshold]
    plt.hist(foo, label='found_injections', bins=100, alpha=0.8)
    noise_stats = noise_stats[noise_stats > lower_threshold]
    plt.hist(noise_stats, label='noise', bins=100, alpha=0.8)
    plt.yscale('log')
    plt.grid(True, which='both')
    plt.legend()
    plt.savefig(os.path.join(output_dir, 'network_output_{}.png'.format(team_name)))
    plt.close()

In [ ]:
for team in [team_1, team_2]:
    
    print('\nTeam {}'.format(team['name']))
    print('Sorting foreground event times')
    sidxs = team["fgevents"][0].argsort()
    fgevents = team["fgevents"].T[sidxs].T
    
    logging.info('Finding injection times closest to event times')
    idxs = find_closest_index(injtimes, fgevents[0])
    diff = np.abs(injtimes[idxs] - fgevents[0])
    
    # If the difference between the injection time and trigger is within tc variance
    # The trigger is identified as an event (there may be duplicate triggers)
    logging.info('Finding true- and false-positives')
    tpbidxs = diff <= fgevents[2]
    tpidxs = np.arange(len(fgevents[0]))[tpbidxs]
    fpbidxs = diff > fgevents[2]
    fpidxs = np.arange(len(fgevents[0]))[fpbidxs]
    
    tpevents = fgevents.T[tpidxs].T
    fpevents = fgevents.T[fpidxs].T
    
    ## Update the returns dictionary
    if team['name'] == "Sage":
        ret['fg-events'] = fgevents
        ret['found-indices'] = np.arange(len(injtimes))[idxs]
        ret['missed-indices'] = np.setdiff1d(np.arange(len(injtimes)), ret['found-indices'])
        ret['true-positive-event-indices'] = tpidxs
        ret['false-positive-event-indices'] = fpidxs
        ret['sorting-indices'] = sidxs
        ret['true-positive-diffs'] = diff[tpidxs]
        ret['false-positive-diffs'] = diff[fpidxs]
        ret['true-positives'] = tpevents
        ret['false-positives'] = fpevents
    
    # Calculate foreground FAR
    logging.info('Calculating foreground FAR')
    noise_stats_fg = fpevents[1].copy()
    noise_stats_fg.sort()
    fgfar = len(noise_stats_fg) - np.arange(len(noise_stats_fg)) - 1
    fgfar = fgfar / duration
    if team['name'] == "Sage":
        ret['fg-far'] = fgfar
    
    # Calculate background FAR
    logging.info('Calculating background FAR')
    noise_stats = team["bgevents"][1].copy()
    noise_stats.sort()
    far = len(noise_stats) - np.arange(len(noise_stats)) - 1
    far = far / duration
    if team['name'] == "Sage":
        ret['far'] = far
    
    # Find best true-positive for each injection
    found_injections = []
    tmpsidxs = idxs.argsort()
    sorted_idxs = idxs[tmpsidxs]
    iidxs = np.full(len(idxs), False)
    for i in tqdm(range(len(injtimes)), ascii=True, desc='Determining found injections'):
        L = np.searchsorted(sorted_idxs, i, side='left')
        if L >= len(idxs) or sorted_idxs[L] != i:
            continue
        R = np.searchsorted(sorted_idxs, i, side='right')
        # All indices that point to the same injection
        iidxs[tmpsidxs[L:R]] = True
        # Indices of the true-positives that belong to the same injection
        eidxs = np.logical_and(iidxs[tmpsidxs[L:R]],
                                tpbidxs[tmpsidxs[L:R]])
        if eidxs.any():
            found_injections.append([i, np.max(fgevents[1][tmpsidxs[L:R]][eidxs])])
        iidxs[tmpsidxs[L:R]] = False

    # Number of injections found within given testing data
    found_injections = np.array(found_injections).T
    print('Number of found injections = {}'.format(len(found_injections[0])))
    
    # Calculate sensitivity
    # CARE! THIS APPLIES ONLY WHEN THE DISTRIBUTION IS CHOSEN CORRECTLY
    logging.info('Calculating sensitivity')
    sidxs = found_injections[1].argsort() # Sort found injections
    found_injections = found_injections.T[sidxs].T
    
    if chirp_distance:
        found_mchirp_total = massc[found_injections[0].astype(int)]
        mchirp_max = massc.max()
        
    max_distance = dist.max()
    # print('Maximum distance given by injections = {}'.format(max_distance))
    vtot = (4. / 3.) * np.pi * max_distance**3.
    Ninj = len(dist)
    print('Total number of injections = {}'.format(Ninj))
    
    # Params to calculate the sensitive volume
    if chirp_distance:
        mc_norm = mchirp_max ** (5. / 2.) * len(massc)
    else:
        mc_norm = Ninj
        
    prefactor = vtot / mc_norm
    nfound = len(found_injections[1]) - np.searchsorted(found_injections[1],
                                                        noise_stats,
                                                        side='right')
    
    
    if chirp_distance:
        # Get found chirp-mass indices for given threshold
        fidxs = np.searchsorted(found_injections[1], noise_stats, side='right')
        # Plotting the network output
        network_output(found_injections, noise_stats, output_dir, team['name'], lower_threshold=-999)
        
        found_mchirp_total = np.flip(found_mchirp_total)
        
        # Calculate sum(found_mchirp ** (5/2))
        # with found_mchirp = found_mchirp_total[i:]
        # and i looped over fidxs
        # Code below is a vectorized form of that
        cumsum = np.flip(np.cumsum(found_mchirp_total ** (5./2.)))
        cumsum = np.concatenate([cumsum, np.zeros(1)])
        mc_sum = cumsum[fidxs]
        Ninj = np.sum((mchirp_max / massc) ** (5. / 2.))
        
        cumsumsq = np.flip(np.cumsum(found_mchirp_total ** 5))
        cumsumsq = np.concatenate([cumsumsq, np.zeros(1)])
        sample_variance_prefactor = cumsumsq[fidxs]
        sample_variance = sample_variance_prefactor / Ninj\
                            - (mc_sum / Ninj) ** 2  # noqa: E127
    else:
        mc_sum = nfound
        sample_variance = nfound / Ninj - (nfound / Ninj) ** 2
        
    vol = prefactor * mc_sum
    vol_err = prefactor * (Ninj * sample_variance) ** 0.5
    rad = (3 * vol / (4 * np.pi))**(1. / 3.)
    print('Radius or sensitive distance as calculated from the volume obtained ({})'.format(team['name']))
    print('min rad = {}, max rad = {}'.format(min(rad), max(rad)))
    
    if team['name'] == "Sage":
        ret['sensitive-volume'] = vol
        ret['sensitive-distance'] = rad
        ret['sensitive-volume-error'] = vol_err
        ret['sensitive-fraction'] = nfound / Ninj
    
    if team['name'] == "PyCBC" or team['name'] == "aresgw":
        ret['sensitive-distance-pycbc'] = rad
        ret['far-pycbc'] = far
    
    # Update plotting params for each group
    team['found_idx'] = found_injections[0].astype(int)
    team['found_stats'] = found_injections[1]
    # Add all found injparams to to plotting dict
    team['params'] = list(injparams.keys())
    team.update(injparams)
    # The values given are indices and have to be 1 less than the number of FA per month req.
    team['far_thresholds'] = noise_stats[::-1][[0, 3, 29, 99, 999]]
    team['noise_stats'] = noise_stats[::-1]
    if team['name'] == 'Sage' and False:
        with open('noise_stats.pickle', 'wb') as handle:
            pickle.dump(team['noise_stats'], handle, protocol=pickle.HIGHEST_PROTOCOL)
    team['sens_dist'] = rad
    team['sens_frac'] = nfound / Ninj

In [ ]:
team_1['far_thresholds']

### Found/Missed Corner Plot

In [ ]:
# Colour table
# orange/purple - "#EB6123", "#512888"
# blue/red - "#2F67B1", "#BF2C23"
# pale blue - "#3A93C3", dark blue - "#1065AB"
# pale red - "#D75F4C", dark red - "#B31529"
# pale green - "#5CAE63", dark green - "#1B7939"
# Super light blue/green - "#D1E5F0", "#D9F1D5"
# yellow/blue - "#FDB338", "#025196"
# dark purple - "#742881"

# Ross color scale - ["#000000","#6db6ff","#009292","#ff6db6","#924900","#490092","#006ddb","#b66dff","#004949","#b6dbff","#920000","#ffb6db","#db6d00","#24ff24","#ffff6d"]
# Plasma perceptually uniform - cmap = plt.cm.plasma
# "#F57D15", "#65156E"

dark_purple = "#742881"
dark_green = "#1B7939"
dark_blue = "#1065AB"
dark_red = "#BF2C23"
taupe = "#B9A281"
sage_green = "#0A5C36"
terracotta = "#CB6843"
light_terracotta = "#DA957B"
raspberry = "#E30B5C"
brown = "#A68A64"
dark_brown = "#855e46"
yellow = "#FDB338"
blue = "#025196"

# Nice plasma
# colors_histogram = ["#D44842", "#65156E"]
# colors_scatter = ["#D44842", "#65156E"]

# cmap colors
choose_cmap = mpl.colormaps['turbo']
hex_left = mpl.colors.rgb2hex(choose_cmap(0.05), keep_alpha=True)
hex_right = mpl.colors.rgb2hex(choose_cmap(0.95), keep_alpha=True)

colors_histogram = [hex_left, hex_right]

# blue --> yellow
# colors_histogram = ["#DAA520", "#00346e"] # yellow/blue
# colors_histogram = ["#936b09", "#00346e"]
# colors_scatter = ["#DAA520", "#00346e"]

In [ ]:
coord_to_name = {}
coord_to_name[0] = 'mchirp'
coord_to_name[1] = 'q'
coord_to_name[2] = 'distance'
coord_to_name[3] = 'snr'

In [ ]:
name_to_label = {}
name_to_label['distance'] = r'$D$'
name_to_label['snr'] = r'$\rho$'
name_to_label['q'] = r'$q$'
name_to_label['mchirp'] = r'$\mathcal{M}_c$'

In [ ]:
thresh_names = ['1-per-month', '1-per-week', '1-per-day', '100-per-month', '1000-per-month']
# How many signals are present above given threshold?
for n, thresh in enumerate(team_1["far_thresholds"]):
    team_1[thresh_names[n]] = team_1['found_idx'][team_1["found_stats"] > thresh]
for n, thresh in enumerate(team_2["far_thresholds"]):
    team_2[thresh_names[n]] = team_2['found_idx'][team_2["found_stats"] > thresh]

In [ ]:
thresh_name = '1-per-month'

In [ ]:
plotting_params = ['mchirp', 'q', 'distance', 'snr']
data_pycbc = {}
data_sage = {}
for param in plotting_params:
    data_sage[param] = team_1[param][team_1[thresh_name]]
    data_pycbc[param] = team_2[param][team_2[thresh_name]]

In [ ]:
# hue_neg, hue_pos = 260, 15
hue_neg, hue_pos = 260, 15
cmap_diverging_dark = sns.diverging_palette(hue_neg, hue_pos, sep=1, center="dark", as_cmap=True)
print(cmap_diverging_dark) # this is a matplotlib object

In [ ]:
cmap_segmented = cmap_diverging_dark  # define the colormap
# extract all colors from the .jet map
cmaplist = [cmap_segmented(i) for i in range(cmap_segmented.N)]

# create the new map
cmap_segmented = mpl.colors.LinearSegmentedColormap.from_list(
            'Custom cmap', cmaplist, cmap_segmented.N)

# define the bins and normalize
# Should have -1 and 1 but not 0
# pycbc - bounds = np.linspace(-11, 11, 12)
bounds = np.arange(-22, 24, 2)
print(bounds)
norm = mpl.colors.BoundaryNorm(bounds, cmap_segmented.N)

In [ ]:
ListedColormap(cmap_diverging_dark(np.linspace(0.0, 0.50, 10)))

In [ ]:
ListedColormap(cmap_diverging_dark(np.linspace(0.50, 1.0, 10)))

In [ ]:
ListedColormap(cmap_diverging_dark(np.linspace(0.0, 1.0, 10)))

In [ ]:
ListedColormap(cmap_diverging_dark(np.linspace(0.0, 1.0, 21)))

In [ ]:
def reduce_function(z):
    z = np.array(z)
    num_pycbc = len(z[z == 0])
    num_sage = len(z[z == 1])
    return num_sage-num_pycbc

In [ ]:
x_data = np.hstack((data_pycbc['mchirp'], data_sage['mchirp']))
y_data = np.hstack((data_pycbc['q'], data_sage['q']))
Z = np.hstack((np.full(len(data_pycbc['mchirp']), 0), np.full(len(data_sage['mchirp']), 1)))
hexb = plt.hexbin(x_data, y_data, C=Z, reduce_C_function=reduce_function, gridsize=32)
plt.close()

positions = hexb.get_offsets()
values = hexb.get_array()

upper_hexbin = int(max(values))
lower_hexbin = int(min(values))

In [ ]:
# Create custom colormap based on limits provided
# We need an odd number of colors to include zero difference
## CHANGE LIMITS HERE!!!
limits = (lower_hexbin, upper_hexbin)
lower = np.linspace(0.0, 0.40, abs(limits[0]))
upper = np.linspace(0.60, 1.0, abs(limits[1]))
full = np.concatenate((lower, [0.5], upper))
print(full)

required = full[:]
ListedColormap(cmap_diverging_dark(required))

In [ ]:
custom_diverging_cmap = ListedColormap(cmap_diverging_dark(required))

In [ ]:
colors1 = plt.cm.viridis_r(np.linspace(0.0, 0.5, abs(limits[0])))
colors2 = plt.cm.plasma(np.linspace(0.0, 0.5, abs(limits[1])))
black = cmap_diverging_dark([0.5])

# combine them and build a new colormap
colors = np.vstack((colors1, black, colors2))
custom = mcolors.LinearSegmentedColormap.from_list('my_colormap', colors)
ListedColormap(custom(np.linspace(0, 1, 28)))

In [ ]:
custom_perceptually_uniform = ListedColormap(custom(np.linspace(0, 1, 28)))

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 6))
x_data = np.hstack((data_pycbc['mchirp'], data_sage['mchirp']))
y_data = np.hstack((data_pycbc['q'], data_sage['q']))
bins = np.histogram(np.hstack((data_pycbc['mchirp'], data_sage['mchirp'])), bins=40)[1]
ax.hist(data_pycbc['mchirp'], density=False, bins=bins, alpha=0.8, histtype='step', color=colors_histogram[0], linewidth=2.0, linestyle='dashed', label='PyCBC')
ax.hist(data_sage['mchirp'], density=False, bins=bins, alpha=0.8, histtype='step', color=colors_histogram[1], linewidth=2.0, linestyle='solid', label='Sage')

plt.xlabel('Chirp Mass (in Solar mass)')
plt.ylabel('Number of Detected Signals')
plt.legend()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 6))
x_data = np.hstack((data_pycbc['mchirp'], data_sage['mchirp']))
y_data = np.hstack((data_pycbc['q'], data_sage['q']))
Z = np.hstack((np.full(len(data_pycbc['mchirp']), 0), np.full(len(data_sage['mchirp']), 1)))
# rf = partial(reduce_function, arg=arg)
# hexb = plt.hexbin(x_data, y_data, C=Z, reduce_C_function=reduce_function, gridsize=32, cmap=cmap_segmented, vmin=-5, vmax=22)
# hexb = plt.hexbin(x_data, y_data, C=Z, reduce_C_function=reduce_function, gridsize=32, cmap=cmap_segmented, norm=norm)
hexb = plt.hexbin(x_data, y_data, C=Z, reduce_C_function=reduce_function, gridsize=32, cmap=custom_diverging_cmap, norm=mpl.colors.Normalize(vmin=lower_hexbin-0.5,vmax=upper_hexbin+0.5))
cbar = plt.colorbar(hexb, ticks=np.arange(lower_hexbin, upper_hexbin+1, 1))
cbar.ax.tick_params(labelsize=12)
plt.xlabel(r'$\mathcal{M}_c$')
plt.ylabel(r'$q$')

positions = hexb.get_offsets()
values = hexb.get_array()

print(max(values), min(values))

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(8, 7.5))
gs = gridspec.GridSpec(2, 2, width_ratios=[4,1], height_ratios=[1,4], wspace=0, hspace=0)

# 2D hexbin (main)
ax_hex = plt.subplot(gs[2])
x_data = np.hstack((data_pycbc['mchirp'], data_sage['mchirp']))
y_data = np.hstack((data_pycbc['q'], data_sage['q']))
Z = np.hstack((np.full(len(data_pycbc['mchirp']), 0), np.full(len(data_sage['mchirp']), 1)))
hexb = ax_hex.hexbin(x_data, y_data, C=Z, reduce_C_function=reduce_function, gridsize=32, cmap=custom_diverging_cmap, norm=mpl.colors.Normalize(vmin=lower_hexbin-0.5,vmax=upper_hexbin+0.5))
ax_hex.set_xlabel(r'$\mathcal{M}_c\;(\mathregular{in}\;\mathregular{M}_\odot)$')
ax_hex.set_ylabel(r'$q$')

ax_hex.xaxis.set_minor_locator(AutoMinorLocator())
ax_hex.yaxis.set_minor_locator(AutoMinorLocator())
ax_hex.tick_params(axis="x",which='minor',length=2.5,bottom=True,left=True,top=True,right=True,direction='in')
ax_hex.tick_params(axis="y",which='minor',length=2.5,bottom=True,left=True,top=True,right=True,direction='in')
ax_hex.tick_params(axis="x",which='major',length=5,bottom=True,left=True,top=True,right=True,direction='in')
ax_hex.tick_params(axis="y",which='major',length=5,bottom=True,left=True,top=True,right=True,direction='in')

# 1D histogram (dim x)
ax_dim1 = plt.subplot(gs[0])
bins = np.histogram(np.hstack((data_pycbc['mchirp'], data_sage['mchirp'])), bins=40)[1]
ax_dim1.hist(data_pycbc['mchirp'], density=False, bins=bins, alpha=0.8, histtype='step', color=colors_histogram[0], linewidth=1.5, linestyle='dashed')
ax_dim1.hist(data_sage['mchirp'], density=False, bins=bins, alpha=1.0, histtype='step', color=colors_histogram[1], linewidth=1.5, linestyle='solid')
# Cleanin up 1D histogram subplots
ax_dim1.set_xticklabels([])
ax_dim1.set_yticklabels([])
ax_dim1.tick_params(bottom=False, left=False, right=False, top=False)
ax_dim1.spines['top'].set_visible(False)
ax_dim1.spines['right'].set_visible(False)
ax_dim1.spines['bottom'].set_visible(False)
ax_dim1.spines['left'].set_visible(False)

# 1D histogram (dim y)
ax_dim2 = plt.subplot(gs[3])
bins = np.histogram(np.hstack((data_pycbc['q'], data_sage['q'])), bins=40)[1]
ax_dim2.hist(data_pycbc['q'], density=False, bins=bins, alpha=0.8, histtype='step', color=colors_histogram[0], linewidth=1.5, linestyle='dashed', orientation='horizontal')
ax_dim2.hist(data_sage['q'], density=False, bins=bins, alpha=1.0, histtype='step', color=colors_histogram[1], linewidth=1.5, linestyle='solid', orientation='horizontal')
# Cleanin up 1D histogram subplots
ax_dim2.set_xticklabels([])
ax_dim2.set_yticklabels([])
ax_dim2.tick_params(bottom=False, left=False, right=False, top=False)
ax_dim2.spines['top'].set_visible(False)
ax_dim2.spines['right'].set_visible(False)
ax_dim2.spines['bottom'].set_visible(False)

# Legend for 1D histograms
sage_line = mlines.Line2D([], [], color=colors_histogram[1], label='Sage', linewidth=2.0)
pycbc_line = mlines.Line2D([], [], color=colors_histogram[0], label='PyCBC', linewidth=2.0, linestyle='dashed')
# left/right, up/down, width, height
plt.legend(handles=[sage_line, pycbc_line], bbox_to_anchor=(-1.8, 0.6, 0.5, 0.5), loc='center left', frameon=False, handlelength=2.6)

# Other plotting params
# left/right, up/down, width, height
cax = ax_hex.inset_axes([5.0, -0.8, 40.0, 0.4], transform=ax_hex.transData)
cbar = fig.colorbar(hexb, cax=cax, orientation='horizontal', ticks=np.arange(lower_hexbin, upper_hexbin+1, 1.0))
cbar.set_label(r'$\mathregular{N^F_{sage} - N^F_{pycbc}}$')
cbar.ax.tick_params(labelsize=12)
tick_rotation = 45
plt.setp(cbar.ax.get_xticklabels(), rotation=tick_rotation, horizontalalignment='right')

plt.savefig("./evaluation_plots/compare_hexbinned_2params.png", bbox_inches='tight', dpi=500)

positions = hexb.get_offsets()
values = hexb.get_array()

In [ ]:
# Should include a zero
# pycbc - np.linspace(-12.0, 12.0, 13)
np.arange(-5.0, 24.0, 2.0)

In [ ]:
# create figure and axes
num_rows = len(data_sage.keys())
num_cols = len(data_sage.keys())
fig, axes = plt.subplots(num_rows, num_cols, figsize=(4*num_rows, 4*num_cols), facecolor='white')
gs = gridspec.GridSpec(num_rows, num_cols, wspace=0, hspace=0)
tick_rotation = 45.0 # degrees
flag = True
    
coords = itertools.product(np.arange(num_rows), np.arange(num_cols))
names = itertools.product(list(data_sage.keys()), list(data_sage.keys()))
for name, coord in zip(names, coords):
    param_1, param_2 = name
    if coord[0] == coord[1]:
        assert param_1 == param_2
        paxis = plt.subplot(gs[int(4*coord[0]+coord[1])])
        ax = paxis
        # Make histogram
        bins = np.histogram(np.hstack((data_pycbc[param_1], data_sage[param_1])), bins=40)[1]
        ax.hist(data_pycbc[param_1], density=False, bins=bins, alpha=0.8, histtype='step', color=colors_histogram[0], linewidth=2.0, linestyle='dashed')
        ax.hist(data_sage[param_1], density=False, bins=bins, alpha=1.0, histtype='step', color=colors_histogram[1], linewidth=2.0, linestyle='solid')

        plt.setp(ax.get_xticklabels(), rotation=tick_rotation, horizontalalignment='right')
        plt.setp(ax.get_yticklabels(), rotation=tick_rotation, horizontalalignment='right')

        if coord[1] == 0:
            ax.set_xticklabels([])
            ax.set_yticklabels([])
        elif coord[0] == num_cols-1:
            paxis.set_xlabel(name_to_label[coord_to_name[coord[1]]])
            ax.set_yticklabels([])
            ax.set_yticks([])
        else:
            ax.set_xticklabels([])
            ax.set_yticklabels([])
            ax.set_xticks([])
            ax.set_yticks([])

    elif coord[1] > coord[0]:
        ax = plt.subplot(gs[int(4*coord[0]+coord[1])])
        ax.remove()
        continue

    else:
        ppaxis = plt.subplot(gs[int(4*coord[0]+coord[1])])
        # marker = '+' if n==0 else 'o'
        # s = 20 if n==0 else 25
        # kwargs = {'color':colors_scatter[n]} if n==0 else {'edgecolors':colors_scatter[n], 'facecolors':'none'}
        # ppaxis.scatter(p2_data, p1_data, s=s, alpha=0.8, marker=marker, **kwargs)

        ##  Replacing the scatter plot with hexbinned color plot
        x_data = np.hstack((data_pycbc[coord_to_name[coord[0]]], data_sage[coord_to_name[coord[0]]]))
        y_data = np.hstack((data_pycbc[coord_to_name[coord[1]]], data_sage[coord_to_name[coord[1]]]))
        Z = np.hstack((np.full(len(data_pycbc[coord_to_name[coord[0]]]), 0), np.full(len(data_sage[coord_to_name[coord[0]]]), 1)))
        hexb = plt.hexbin(y_data, x_data, C=Z, reduce_C_function=reduce_function, gridsize=32, cmap=custom_diverging_cmap, norm=mpl.colors.Normalize(vmin=lower_hexbin-0.5,vmax=upper_hexbin+0.5))
        values = hexb.get_array()
        print(min(values), max(values))

        if flag:
            # left/right, up/down, width, height
            cax = ppaxis.inset_axes([150.0, 5.0, 3.0, 7.0], transform=ppaxis.transData)
            cbar = fig.colorbar(hexb, cax=cax, orientation='vertical', ticks=np.arange(lower_hexbin, upper_hexbin+1, 1.0))
            cbar.set_label(r'$\mathregular{N^F_{sage} - N^F_{pycbc}}$', rotation=90)
            cbar.ax.tick_params(labelsize=12)
            flag = False

        name = 'PyCBC' if n==0 else 'Sage'
        """
        corner.hist2d(p2_data, p1_data,
                    ax=ppaxis, color=colors_fill[n],
                    levels=[0.95, 0.68], bins=512, smooth=42.,
                    plot_datapoints=False, plot_contours=True, fill_contours=False,
                    plot_density=False, contour_kwargs=contour_dict[name])
        """
        # Rotate tick labels
        plt.setp(ppaxis.get_xticklabels(), rotation=tick_rotation, horizontalalignment='right')
        plt.setp(ppaxis.get_yticklabels(), rotation=tick_rotation, horizontalalignment='right')

        if coord[1] == 0:
            ppaxis.set_ylabel(name_to_label[param_1])
            if coord[0] != num_rows - 1:
                ppaxis.set_xticklabels([])
                ppaxis.set_xticks([])
        if coord[0] == num_cols-1:
            ppaxis.set_xlabel(name_to_label[param_2])
            if coord[1] != 0:
                ppaxis.set_yticklabels([])
                ppaxis.set_yticks([])
        if coord[1] !=0 and coord[0] != num_cols-1:
            ppaxis.set_xticklabels([])
            ppaxis.set_yticklabels([])
            ppaxis.set_xticks([])
            ppaxis.set_yticks([])

## save the plot
fig.subplots_adjust(wspace=0)
fig.subplots_adjust(hspace=0)
# Legend
sage_line = mlines.Line2D([], [], color=colors_histogram[1], label='Sage', linewidth=4.0)
pycbc_line = mlines.Line2D([], [], color=colors_histogram[0], label='PyCBC', linewidth=4.0, linestyle='dashed')
# left/right, up/down, width, height
plt.legend(handles=[sage_line, pycbc_line], bbox_to_anchor=(-0.45, 0.33, 1., 6.5), loc='center left', frameon=False, fontsize=22, handlelength=2.6)
# Found and missed numbers
sage_set = set(data_sage['mchirp'])
pycbc_set = set(data_pycbc['mchirp'])
found_by_both = len(set.intersection(sage_set, pycbc_set))
found_only_by_sage = len(sage_set - pycbc_set)
found_only_by_pycbc = len(pycbc_set - sage_set)
plt.text(0.35, 3.2, 'Found by both = {}'.format(str(found_by_both)), transform=ax.transAxes, horizontalalignment='right', fontsize=15)
plt.text(0.35, 3.2-0.125, 'Found only by Sage = {}'.format(str(found_only_by_sage)+'  '), transform=ax.transAxes, horizontalalignment='right', fontsize=15)
plt.text(0.35, 3.2-0.250, 'Found only by PyCBC = {}'.format(str(found_only_by_pycbc)+'  '), transform=ax.transAxes, horizontalalignment='right', fontsize=15)

plt.savefig("./evaluation_plots/compare_hexbinned.png", bbox_inches='tight', dpi=500)

### Making found/missed histograms at different FARs

In [ ]:
save_dir = os.path.join(output_dir, "histogram")

# Plot 1 (Histogram of all injections with found injections of both pipelines)
os.makedirs(save_dir, exist_ok=False)
# params = team_1['params']
params = ['mass1', 'mass2', 'mchirp',
          'q', 'distance', 'chirp_distance',
          'coa_phase', 'inclination', 'polarization',
          'ra', 'dec', 'snr',
          'spin1x', 'spin1y', 'spin1z',
          'spin2x', 'spin2y', 'spin2z']

ncols = 3
nrows = len(params)//ncols + int(len(params)%ncols or False)

thresh_names = ['1-per-month', '1-per-week', '1-per-day', '100-per-month', '1000-per-month']
# How many signals are present above given threshold?
for n, thresh in enumerate(team_1["far_thresholds"]):
    team_1[thresh_names[n]] = team_1['found_idx'][team_1["found_stats"] > thresh]
for n, thresh in enumerate(team_2["far_thresholds"]):
    team_2[thresh_names[n]] = team_2['found_idx'][team_2["found_stats"] > thresh]

for thresh_name in thresh_names:
    # Subplotting
    fig, ax = plt.subplots(nrows, ncols, figsize=(8.0*ncols, 6.0*nrows))
    # Histogram kwargs
    kwargs = dict(histtype='stepfilled', alpha=0.5)
    
    pidxs = list(itertools.product(range(nrows), range(ncols)))
    num_fin = 0
    for param, (i, j)  in zip(params, pidxs):
        bins = np.histogram(np.hstack((team_2[param][team_2[thresh_name]], team_1[param][team_1[thresh_name]])), bins=40)[1]
        ax[i][j].hist(team_2[param][team_2[thresh_name]], bins=bins, alpha=0.8, histtype='step', color=colors_histogram[0], linewidth=2.0, linestyle='dashed')
        ax[i][j].hist(team_1[param][team_1[thresh_name]], bins=bins, alpha=1.0, histtype='step', color=colors_histogram[1], linewidth=2.0, linestyle='solid')
        ax[i][j].set_title(param)
        ax[i][j].grid(True)
        ax[i][j].legend()
        num_fin+=1
    
    for i, j in pidxs[num_fin:]:
        ax[i][j].set_visible(False)
    
    plt.tight_layout()
    save_name = "compare_histogram_{}_and_{}-{}.png".format(team_1['name'], team_2['name'], thresh_name)
    save_path = os.path.join(save_dir, save_name)
    plt.savefig(save_path)
    plt.close()

### Sensitive Distance Plot

In [ ]:
# Store results
stats = ret
with h5py.File("./evaluation_plots/evaluation_output.hdf", 'w') as fp:
    for key, val in stats.items():
        fp.create_dataset(key, data=np.array(val))

# Create the sensitivity vs FAR/month plot from the output evaluation obtained
assert dur == far_scaling_factor, 'FAR scaling factor discrepancy! Check duration.'
with h5py.File("./evaluation_plots/evaluation_output.hdf", 'r') as fp:
    far = fp['far'][()]
    sens = fp['sensitive-distance'][()]
    sidxs = far.argsort()
    far = far[sidxs][1:] * far_scaling_factor
    sens = sens[sidxs][1:]
    
    far_pycbc = fp['far-pycbc'][()]
    sidxs_pycbc = far_pycbc.argsort()
    far_pycbc_chk = far_pycbc[sidxs_pycbc] * far_scaling_factor
    sens_pycbc_check = fp['sensitive-distance-pycbc'][()]
    sens_pycbc_check = sens_pycbc_check[sidxs_pycbc]

# Month FAR factor
month = 30.0 * 24.0 * 60.0 * 60.0

In [ ]:
plt.figure(figsize=(12.0, 8.0))
plt.title('Sensitivity Measure for Dataset {}'.format(dataset))
plt.plot(far*(month/dur), sens, color=sage_green, linewidth=2.5, label='Sage')

with h5py.File('/home/nnarenraju/Research/ORChiD/results/PyCBC/ds{}/eval.hdf'.format(dataset)) as fp:
    sens_pycbc = np.array(fp['sensitive-distance'])
    far_pycbc = np.array(fp['far'])
plt.plot(far_pycbc*month, sens_pycbc, color=dark_brown, linewidth=2.0, linestyle='dashed', label='PyCBC')

with h5py.File('/home/nnarenraju/Research/ORChiD/results/TPI_FSU_Jena/ds{}/eval.hdf'.format(dataset)) as fp:
    sens_fsu = np.array(fp['sensitive-distance'])
    far_fsu = np.array(fp['far'])
plt.plot(far_fsu*month, sens_fsu, color='red', linewidth=2.0, linestyle='dashed', label='TPI FSU Jena')

with h5py.File('/home/nnarenraju/Research/ORChiD/results/Virgo-AUTh/ds{}/eval.hdf'.format(dataset)) as fp:
    sens_virgo = np.array(fp['sensitive-distance'])
    far_virgo = np.array(fp['far'])
plt.plot(far_virgo*month, sens_virgo, color='blueviolet', linewidth=2.0, linestyle='dashed', label='Virgo-AUTh')

with h5py.File('/home/nnarenraju/Research/ORChiD/results/CNN-Coinc/ds{}/eval.hdf'.format(dataset)) as fp:
    sens_cnn = np.array(fp['sensitive-distance'])
    far_cnn = np.array(fp['far'])
plt.plot(far_cnn*month, sens_cnn, color='m', linewidth=2.0, linestyle='dashed', label='CNN-Coinc')

with h5py.File('/home/nnarenraju/Research/ORChiD/results/MFCNN/ds{}/eval.hdf'.format(dataset)) as fp:
    sens_mfcnn = np.array(fp['sensitive-distance'])
    far_mfcnn = np.array(fp['far'])
plt.plot(far_mfcnn*month, sens_mfcnn, color='orange', linewidth=2.0, linestyle='dashed', label='MFCNN')

plt.grid(True, which='both')
plt.xlim(1000, 1)
plt.ylim(0, 2500)
plt.xscale('log')
plt.xlabel('False Alarm Rate (FAR) per month')
plt.ylabel('Sensitive Distance [MPc]')
plt.legend()
plt.savefig("./evaluation_plots/sensitive_distance_all_teams.png", bbox_inches='tight', dpi=300)

In [ ]:
plt.figure(figsize=(12.0, 8.0))
# "#D44842", "#65156E"
plt.plot(far*(month/dur), sens, color="#65156E", linewidth=3.0, label='Sage')

with h5py.File('/home/nnarenraju/Research/ORChiD/results/{}/ds{}/eval.hdf'.format('PyCBC', dataset)) as fp:
    sens_team2 = np.array(fp['sensitive-distance'])
    far_team2 = np.array(fp['far'])
    plt.plot(far_team2*month, sens_team2, color="#D44842", linewidth=2.0, linestyle='dashed', label='{}'.format('PyCBC'))

plt.grid(True, which='both')
plt.xlim(1000, 1)
plt.ylim(1500, 2100)
plt.xscale('log')
plt.xlabel('False Alarm Rate (FAR) per month')
plt.ylabel('Sensitive Distance [MPc]')
plt.legend()
plt.savefig("./evaluation_plots/sensitive_distance_pycbc_sage.png", bbox_inches='tight', dpi=300)


### ROC Curve Comparison

In [ ]:
def figure(title=""):
    # plt.rc('font', family='serif')
    plt.rc('xtick', labelsize='medium')
    plt.rc('ytick', labelsize='medium')
    fig, axs = plt.subplots(1, figsize=(16.0, 14.0))
    fig.suptitle(title, fontsize=18, y=0.92)
    return axs, fig


def _plot(ax, x=None, y=None, xlabel="x-axis", ylabel="y-axis", ls='solid', 
          label="NULL", c=None, yscale='linear', xscale='linear', histogram=False):
    
    # Plotting type
    if histogram:
        ax.hist(y, bins=100, label=label, alpha=0.8)
    else:
        ax.plot(x, y, ls=ls, c=c, linewidth=3.0, label=label)
    
    # Plotting params
    ax.set_xscale(xscale)
    ax.set_yscale(yscale)
    ax.grid(True, which='both')
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    ax.legend()

In [ ]:
def read_network_output(fgpath, bgpath):
    with h5py.File(fgpath, 'r') as fp:
        fg_events = [np.vstack([fp['time'], fp['stat'], fp['var']])]
    fg_events = np.concatenate(fg_events, axis=-1)
    sidxs = fg_events[0].argsort()
    fg_events = fg_events.T[sidxs].T
    
    # Read background events
    with h5py.File(bgpath, 'r') as fp:
        bg_events = [np.vstack([fp['time'], fp['stat'], fp['var']])]
    bg_events = np.concatenate(bg_events, axis=-1)
    sidxs = bg_events[0].argsort()
    bg_events = bg_events.T[sidxs].T

    return fg_events, bg_events

In [ ]:
## Foreground and background dataset outputs from pipeline
teams = {'PyCBC': {'fg_events': None, 'bg_events': None}, 
         'Sage': {'fg_events': None, 'bg_events': None}}

# Paths
# fgpath = ["/home/nnarenraju/Research/ORChiD/results/PyCBC/ds4/fg.hdf", foutput]
# bgpath = ["/home/nnarenraju/Research/ORChiD/results/PyCBC/ds4/bg.hdf", boutput]
fout = foutput
bout = boutput
fgpath = ["/home/nnarenraju/Research/ORChiD/results/PyCBC/ds4/fg.hdf", fout]
bgpath = ["/home/nnarenraju/Research/ORChiD/results/PyCBC/ds4/bg.hdf", bout]
# Get data from the outputs files
for fgp, bgp, tname in zip(fgpath, bgpath, teams.keys()):
    fg_events, bg_events = read_network_output(fgp, bgp)
    teams[tname]['bg_events'] = bg_events
    teams[tname]['fg_events'] = fg_events

## Injections file and foreground file data
injfile = injections
fgfile = foreground

In [ ]:
# Find all valid injection times for the given testing dataset
duration, valid_idxs = find_injection_times([fgfile], injfile)
# Read the injections file and get the values of injection params
print("Reading the injections file for testing dataset")
assert all(team_1['tc'] == team_2['tc'])
tc = team_1['tc']

In [ ]:
def find_nclosest_index(trigger_times, injection_times, frange=[-0.3, +0.3]):
    # Find nclosest triggers given an injection time
    if len(trigger_times) == 0:
        raise ValueError('Cannot find closest index for empty trigger times array.')
    
    # Sort array before proceeding
    array = trigger_times.copy()
    array.sort()
    
    # Find nclosest indices for the given value in array
    tidxs = [np.argwhere((array >= inj + frange[0]) & (array <= inj + frange[1])) for inj in tqdm(injection_times)]
    return tidxs

In [ ]:
ax, _ = figure(title=None)

# Save ROC data
save = {}
save['Sage'] = {'fpr': None, 'tpr': None}
save['PyCBC'] = {'fpr': None, 'tpr': None}

for tname in teams.keys():
    print("\nComputing ROC Curve for {}".format(tname))
    fg_events = teams[tname]['fg_events']
    bg_events = teams[tname]['bg_events']

    ## Outputs and Labels for FG and BG
    outputs = []
    labels = []
    
    """ Foreground file analysis """
    print("\nAnalysing the foreground data")
    # Read the foreground output and get all the trigger times found
    fg_trigger_times = fg_events[0]
    fg_trigger_stats = fg_events[1]

    # Given all the valid injection times present in the injections file
    # get idx of all triggers that are within a bound of the given injection time
    print("Getting closest trigger indices for each valid injection time")
    fg_bound_triggers_idx = find_nclosest_index(trigger_times=fg_trigger_times, injection_times=injparams['tc'], frange=[-0.3, +0.3])
    
    print("Maximising over each set of triggers and obtaining the output and labels for ROC curve")
    ## Get outputs and labels to get ROC curve
    pbar = tqdm(fg_bound_triggers_idx)
    for btidx in pbar:
        pbar.set_description("Maximising FG triggers")
        if len(btidx) > 0:
            # Maximise on these triggers to get the event
            outputs.append(np.max(fg_trigger_stats[btidx]))
        else:
            # If there are no triggers given, then we assign 0.0 stat for event
            outputs.append(-np.inf)
        # Append labels as 1.0 since only GW events are looked at
        labels.append(1.0)
    
    """ Background file analysis """
    print("\nAnalysing the background data")
    # Read the foreground output and get all the trigger times found
    bg_trigger_times = bg_events[0]
    bg_trigger_stats = bg_events[1]
    
    # Given all the valid injection times present in the injections file
    # get idx of all triggers that are within a bound of the given injection time
    print("Getting closest trigger indices for each valid injection time")
    bg_bound_triggers_idx = find_nclosest_index(trigger_times=bg_trigger_times, injection_times=injparams['tc'], frange=[-0.3, +0.3])
    
    print("Maximising over each set of triggers and obtaining the output and labels for ROC curve")
    ## Get outputs and labels to get ROC curve
    pbar = tqdm(bg_bound_triggers_idx)
    for btidx in pbar:
        pbar.set_description("Maximising BG triggers")
        if len(btidx) > 0:
            # Maximise on these triggers to get the event
            outputs.append(np.max(bg_trigger_stats[btidx]))
        else:
            # If there are no triggers given, then we assign 0.0 stat for event
            outputs.append(-np.inf)
        # Append labels as 0.0 since only noise data is looked at
        labels.append(0.0)

    """ Calculating the ROC Curve """
    outputs = np.array(outputs)
    labels = np.array(labels)
    # Convert the outputs from raw outputs to prediction probabilites using sigmoid
    sigmoid = lambda x: 1./(1. + np.exp(-1*x))
    outputs = sigmoid(outputs)
    # Calculate the ROC curve
    fpr, tpr, threshold = metrics.roc_curve(labels, outputs)
    roc_auc = metrics.auc(fpr, tpr)

    # Log ROC Curve
    if tname == "PyCBC":
        colour = 'red'
    elif tname == "Sage":
        colour = "blue"

    # Save data
    save[tname]['fpr'] = fpr
    save[tname]['tpr'] = tpr
    _plot(ax, fpr, tpr, label="{}, AUC = {}".format(tname, np.around(roc_auc, 4)), c=colour, 
        ylabel="True Positive Rate", xlabel="False Positive Rate", 
        yscale='log', xscale='log')

# Other plotting stuff
_plot(ax, [0, 1], [0, 1], label="Random Classifier", c='k', 
    ylabel="True Positive Rate", xlabel="False Positive Rate", 
    ls="dashed", yscale='log', xscale='log')

save_path = "./evaluation_plots/roc_curve.png"
plt.savefig(save_path)
plt.close()

In [ ]:
plt.rcParams.update({
    "font.size": 18
})

In [ ]:
# Plotting
plt.figure(figsize=(12.0, 8.0))
# "#D44842", "#65156E"
plt.loglog(save['Sage']['fpr'], save['Sage']['tpr'], c="#65156E", label='Sage', linewidth=2.0)
plt.loglog(save['PyCBC']['fpr'], save['PyCBC']['tpr'], c="#D44842", label='PyCBC', linewidth=2.0)
plt.plot([0, 1], [0, 1], label='Random Classifier', c='k', linestyle='dashed')
plt.xlabel('False Alarm Probability')
plt.ylabel('True Alarm Probability')
plt.xlim(0.0, 1.0)
plt.ylim(0.0, 1.0)
plt.grid(which='both')
plt.legend(loc='lower right')
save_path = "./evaluation_plots/roc_pycbc_sage.png"
plt.tight_layout()
plt.savefig(save_path, bbox_inches='tight', dpi=300)
plt.show()

### Glitch Rejection Capabilities

In [87]:
glitch_names = ['1400Ripples', '1080Lines', 'Air_Compressor', 'Blip', 'Chirp', 'Extremely_Loud', 'Helix',
                'Koi_Fish', 'Light_Modulation', 'Low_Frequency_Burst', 'Low_Frequency_Lines', 'No_Glitch', 
                'None_of_the_Above', 'Paired_Doves', 'Power_Line', 'Repeating_Blips', 'Scattered_Light', 
                'Scratchy', 'Tomte', 'Violin_Mode', 'Wandering_Line', 'Whistle']
glitch_alias = ['1400R', '1080L', 'AC', 'B', 'C', 'EL', 'H',
                'KF', 'LM', 'LFB', 'LFL', 'NG', 
                'NA', 'PD', 'PL', 'RB', 'SL', 
                'S', 'T', 'VM', 'WL', 'W']

map_to_idx = dict(zip(glitch_names, np.arange(len(glitch_names))))
map_to_names = dict(zip(np.arange(len(glitch_names)), glitch_names))
names_to_alias = dict(zip(glitch_names, glitch_alias))

In [88]:
print(len(glitch_alias))

22


In [89]:
def get_glitch_data(detfile):
    # File containing glitch info H1 O3a
    filename = "/local/scratch/igr/nnarenraju/gwspy/{}.csv".format(detfile)
    gfile = pd.read_csv('{}'.format(filename))
    num_glitches_in_odet = len(gfile)
    print('H1 O3a contains {} glitches'.format(num_glitches_in_odet))
    glitch_probs = gfile.iloc[:, 15:37]
    glitch_types = []
    for i in range(num_glitches_in_odet):
        glitch_types.append(map_to_idx[glitch_probs.iloc[i].idxmax()])

    glitch_gps_times = np.array(gfile['start_time'])
    glitch_duration = np.array(gfile['duration'])
    glitch_snrs = np.array(gfile['snr'])
    return (glitch_gps_times, glitch_duration, glitch_snrs, glitch_types)

In [90]:
_, _, _, glitch_types = get_glitch_data('H1_O3a')

H1 O3a contains 80763 glitches


In [91]:
def glitches_in_testset(glitch_gps_times, glitch_duration, glitch_snrs, glitch_types, segments_csv='./tmp/segments.csv'):
    # We use the first 133 noise segments as testing noise
    # This contains start and end times
    noise_times = pd.read_csv(segments_csv)[133:]
    # Check if a glitch reported in gravity spy is present within the start and end times of the noise segments
    num_present = 0
    glitch_gps_times_in_testset = []
    glitch_durations_in_testset = []
    glitch_snrs_in_testset = []
    glitch_types_in_testset = []
    for glitch_time, gdur, gsnr, gtype in tqdm(zip(glitch_gps_times, glitch_duration, glitch_snrs, glitch_types)):
        is_present = np.any((noise_times['start'] <= glitch_time) & (noise_times['end'] >= glitch_time))
        num_present += is_present
        if is_present:
            glitch_gps_times_in_testset.append(glitch_time)
            glitch_durations_in_testset.append(gdur)
            glitch_snrs_in_testset.append(gsnr)
            glitch_types_in_testset.append(gtype)
    print('There were {} glitches present in the testing data for H1 O3a'.format(num_present))
    return (glitch_gps_times_in_testset, glitch_durations_in_testset, glitch_snrs_in_testset, glitch_types_in_testset)

In [92]:
def find_nclosest_index_for_glitch(trigger_times, injection_times, glitch_duration):
    # Find nclosest triggers given an injection time
    if len(trigger_times) == 0:
        raise ValueError('Cannot find closest index for empty trigger times array.')
    
    # Sort array before proceeding
    array = trigger_times.copy()
    array.sort()
    
    # Find nclosest indices for the given value in array
    tidxs = []
    for inj, gdur in zip(injection_times, glitch_duration):
        # tidxs.append(np.argwhere((array >= inj + (-1*gdur)) & (array <= inj + gdur)))
        tidxs.append(np.argwhere((array >= inj) & (array <= inj+gdur)))
    return tidxs

In [93]:
# How close was the peak glitch time to a GW injection
def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return abs(array[idx] - value)

In [94]:
single_trigger = True

In [95]:
def get_relevant_glitches(tname, glitch_gps_times_in_testset, glitch_duration, glitch_snrs_in_testset, glitch_types_in_testset):
    # Get the event time for glitches and get triggers around this GPS time
    # This procedure is similar to what we already have with the ROC curve code
    # Plot all network outputs without maximising against all noise triggers
    bg_events = teams[tname]['bg_events']

    """ Triggers around glitch time """
    # Read the background output and get all the trigger times found
    bg_trigger_times = bg_events[0]
    bg_trigger_stats = bg_events[1]

    # Given all the valid injection times present in the injections file
    # get idx of all triggers that are within a bound of the given injection time
    print("Getting closest trigger indices for each valid injection time")
    bg_bound_triggers_idx = find_nclosest_index_for_glitch(trigger_times=bg_trigger_times, injection_times=glitch_gps_times_in_testset, glitch_duration=glitch_duration)

    foo = team_1 if tname=='Sage' else team_2
    thresh_vals = dict(zip(['1-per-month', '1-per-week', '1-per-day', '100-per-month', '1000-per-month'], foo['far_thresholds']))
    print(thresh_vals)

    # Get glitch data
    glitch_stat_snr = []
    for btidx, gsnr, gtime, gtype in zip(bg_bound_triggers_idx, glitch_snrs_in_testset, glitch_gps_times_in_testset, glitch_types_in_testset):
        if not single_trigger:
            triggers = bg_trigger_stats[btidx]
            glitch_data = np.column_stack((triggers, np.full(len(triggers), gsnr), np.full(len(triggers), gtime), np.full(len(triggers), gtype)))
        else:
            if len(btidx) != 0:
                triggers = [np.max(bg_trigger_stats[btidx])]
            else:
                triggers = bg_trigger_stats[btidx]
            glitch_data = np.column_stack((triggers, np.full(len(triggers), gsnr), np.full(len(triggers), gtime), np.full(len(triggers), gtype)))
        glitch_stat_snr.append(glitch_data)

    # Combine all triggers
    glitch_stat_snr = np.row_stack((glitch_stat_snr))
    # Pick glitch stats that are above the FAR threshold for 1000/month
    ridxs = np.argwhere((glitch_stat_snr[:,0] > thresh_vals['1000-per-month']) & (glitch_types_in_testset != 'No_Glitch')).flatten()
    # ridxs = np.argwhere(glitch_stat_snr[:,0] > thresh_vals['1000-per-month']).flatten()
    # glitches_above_1000_per_month = glitch_stat_snr[ridxs]
    glitches_above_1000_per_month = glitch_stat_snr # use this for violin plot stuff

    # glitches_above_1000_per_month = glitch_stat_snr
    print('Number of glitches above an FAR of 1000/month = {}'.format(glitches_above_1000_per_month.shape[0]))

    nearest_event_dt = []
    for gtime in glitches_above_1000_per_month[:,2]:
        nearest_event_dt.append(find_nearest(team_1['tc'], gtime))
    
    return (glitches_above_1000_per_month, thresh_vals, nearest_event_dt)

In [ ]:
detfiles = ['H1_O3a', 'L1_O3a']
team_names = ['Sage_ub', 'Sage', 'PyCBC']
coords = itertools.product(team_names, detfiles)

fig, axs = plt.subplots(3, 2, figsize=(13.0*2, 9.0*3))
axs = axs.flatten()

save_glitch_data = {'Sage':{'H1_O3a':None, 'L1_O3a':None}, 'PyCBC':{'H1_O3a':None, 'L1_O3a':None}}
tmp = {'Sage_ub':{'H1_O3a':None, 'L1_O3a':None}}

for num, ((tname, detfile), ax) in enumerate(zip(coords, axs)):
    print('\nCurrent coord: {}, {}'.format(detfile, tname))
    if tname == 'Sage_ub':
        ax.set_ylabel('Ranking Statistic ({}, {})'.format('Sage', detfile.replace('_', ' ')), fontsize=22)
    else:
        ax.set_ylabel('Ranking Statistic ({}, {})'.format(tname, detfile.replace('_', ' ')), fontsize=22)
    ax.set_xlabel('Glitch SNR', fontsize=22)
        
    if tname == 'Sage_ub':
        with open('thresh_vals_{}.pickle'.format(detfile), 'rb') as handle:
            thresh_vals = pickle.load(handle)
        with open('xmin_{}.pickle'.format(detfile), 'rb') as handle:
            xmin = pickle.load(handle)
        with open('xmax_{}.pickle'.format(detfile), 'rb') as handle:
            xmax = pickle.load(handle)
        with open('glitch_above_1000_{}.pickle'.format(detfile), 'rb') as handle:
            glitches_above_1000_per_month = pickle.load(handle)

        for thresh_name, thresh in thresh_vals.items():
            ax.hlines(y=thresh, xmin=xmin, xmax=xmax, linewidth=2., label=thresh_name, colors='k', zorder=1)
        # Scatter plot of glitch triggers
        foo = ax.scatter(glitches_above_1000_per_month[:,1], glitches_above_1000_per_month[:,0], s=256, alpha=0.8, c="#65156E", edgecolors='k')

        # placing text instead of legend
        for thresh_name, tval in thresh_vals.items():
            factor = 0.9 if detfile == 'L1_O3a' else 0.8
            ax.text(factor*xmax, tval+0.05, '{}'.format(thresh_name), horizontalalignment='right', fontsize=14)
        
        ax.grid(which='both')
        ax.set_xlim(xmin, xmax)
        ax.set_xscale('log')
        ax.tick_params(axis='both', which='major', labelsize=22)
        ax.tick_params(axis='both', which='minor', labelsize=22)

        tmp[tname][detfile] = glitches_above_1000_per_month
        save_glitch_data.update(tmp)

        continue
    
    glitch_gps_times, glitch_duration, glitch_snrs, glitch_types = get_glitch_data(detfile)
    glitch_gps_times_in_testset, glitch_durations_in_testset, glitch_snrs_in_testset, glitch_types_in_testset = glitches_in_testset(glitch_gps_times, glitch_duration, glitch_snrs, glitch_types)
    glitches_above_1000_per_month, thresh_vals, nearest_event_dt = get_relevant_glitches(tname, glitch_gps_times_in_testset, glitch_duration, glitch_snrs_in_testset, glitch_types_in_testset)

    # Save glitch data
    save_glitch_data[tname][detfile] = glitches_above_1000_per_month

    # Plot all glitch triggers alongside the thresh vals
    xmin = min(glitches_above_1000_per_month[:,1]) - 0.1*min(glitches_above_1000_per_month[:,1])
    xmax = max(glitches_above_1000_per_month[:,1]) + 0.1*max(glitches_above_1000_per_month[:,1])
    xmin = 5.0
    xmax = 10_000
    n = len(thresh_vals.keys())

    # Threshold lines
    colors = np.array([raspberry, dark_red, dark_blue, dark_green, dark_purple])[::-1]
    for (thresh_name, thresh), c in zip(thresh_vals.items(), colors):
        ax.hlines(y=thresh, xmin=xmin, xmax=xmax, linewidth=2., label=thresh_name, colors='k', zorder=1)
    # Scatter plot of glitch triggers
    # foo = ax.scatter(glitches_above_1000_per_month[:,1], glitches_above_1000_per_month[:,0], s=256, alpha=0.8, c=nearest_event_dt, cmap='viridis_r', edgecolors='k')
    foo = ax.scatter(glitches_above_1000_per_month[:,1], glitches_above_1000_per_month[:,0], s=256, alpha=0.8, c="#65156E", edgecolors='k')

    # placing text instead of legend
    for thresh_name, tval in thresh_vals.items():
        factor = 0.9 if detfile == 'L1_O3a' else 0.8
        # ax.text(factor*max(glitches_above_1000_per_month[:,1]), tval+0.05, '{}'.format(thresh_name), horizontalalignment='right', fontsize=14)
        ax.text(factor*xmax, tval+0.05, '{}'.format(thresh_name), horizontalalignment='right', fontsize=14)


    if tname == 'Sage' and False:
        with open('thresh_vals_{}.pickle'.format(detfile), 'wb') as handle:
            pickle.dump(thresh_vals, handle, protocol=pickle.HIGHEST_PROTOCOL)
        with open('xmin_{}.pickle'.format(detfile), 'wb') as handle:
            pickle.dump(xmin, handle, protocol=pickle.HIGHEST_PROTOCOL)
        with open('xmax_{}.pickle'.format(detfile), 'wb') as handle:
            pickle.dump(xmax, handle, protocol=pickle.HIGHEST_PROTOCOL)
        with open('glitch_above_1000_{}.pickle'.format(detfile), 'wb') as handle:
            pickle.dump(glitches_above_1000_per_month, handle, protocol=pickle.HIGHEST_PROTOCOL)

    """
    divider = make_axes_locatable(ax)
    cax = divider.append_axes('right', size='5%', pad=0.05)
    cbar = fig.colorbar(foo, cax=cax, orientation='vertical', alpha=1.0)
    if num == 1 or num == 3 or True:
        cbar.set_label(r'$\mathregular{min|T^{g} - T^{inj}_i|}$', rotation=90)
        cbar.set_alpha(1.0)
    """
    ax.grid(which='both')
    ax.set_xlim(xmin, xmax)
    ax.set_xscale('log')
    ax.tick_params(axis='both', which='major', labelsize=22)
    ax.tick_params(axis='both', which='minor', labelsize=22)

save_path = "./evaluation_plots/glitch_triggers.png"
# plt.tight_layout()
left  = 0.05    # the left side of the subplots of the figure
right = 0.1    # the right side of the subplots of the figure
bottom = 0.1   # the bottom of the subplots of the figure
top = 0.1      # the top of the subplots of the figure
wspace = 0.2   # the amount of width reserved for blank space between subplots
hspace = 0.2   # the amount of height reserved for white space between subplots
plt.subplots_adjust(wspace=wspace)
plt.savefig(save_path, bbox_inches='tight', dpi=500)
plt.show()

In [ ]:
# trigger, snr, gps_time, type
save_glitch_data['Sage']['H1_O3a']

In [107]:
# Make segments file for O3b
H1_O3b_csv = pd.read_csv('./O3b/H1_O3b_GPS.txt', names=['start', 'end', 'duration'], header=None, delimiter=' ')
L1_O3b_csv = pd.read_csv('./O3b/L1_O3b_GPS.txt', names=['start', 'end', 'duration'], header=None, delimiter=' ')
H1_O3b_csv.to_csv('./O3b/H1_O3b_GPS.csv', sep=',', encoding='utf-8', header=True)
L1_O3b_csv.to_csv('./O3b/L1_O3b_GPS.csv', sep=',', encoding='utf-8', header=True)
print(H1_O3b_csv)
print(L1_O3b_csv)

          start         end  duration
0    1256663443  1256679418     15975
1    1256682796  1256686818      4022
2    1256688786  1256705153     16367
3    1256710426  1256746767     36341
4    1256746930  1256750703      3773
..          ...         ...       ...
409  1269198850  1269220459     21609
410  1269229591  1269277120     47529
411  1269288626  1269296489      7863
412  1269296653  1269299246      2593
413  1269305892  1269363618     57726

[414 rows x 3 columns]
          start         end  duration
0    1256655666  1256657739      2073
1    1256663958  1256673192      9234
2    1256673691  1256756709     83018
3    1256766682  1256772748      6066
4    1256775368  1256798948     23580
..          ...         ...       ...
347  1269215272  1269220522      5250
348  1269220720  1269271634     50914
349  1269271730  1269285851     14121
350  1269289707  1269296084      6377
351  1269307031  1269363615     56584

[352 rows x 3 columns]


In [108]:
glitch_gps_times, glitch_duration, glitch_snrs, glitch_types = get_glitch_data('H1_O3a')
_, _, _, glitch_types_in_testset_H1_O3a = glitches_in_testset(glitch_gps_times, glitch_duration, glitch_snrs, glitch_types)
glitch_gps_times, glitch_duration, glitch_snrs, glitch_types = get_glitch_data('L1_O3a')
_, _, _, glitch_types_in_testset_L1_O3a = glitches_in_testset(glitch_gps_times, glitch_duration, glitch_snrs, glitch_types)
glitch_gps_times, glitch_duration, glitch_snrs, glitch_types = get_glitch_data('H1_O3b')
_, _, _, glitch_types_in_testset_H1_O3b = glitches_in_testset(glitch_gps_times, glitch_duration, glitch_snrs, glitch_types, segments_csv='./O3b/H1_O3b_GPS.csv')
glitch_gps_times, glitch_duration, glitch_snrs, glitch_types = get_glitch_data('L1_O3b')
_, _, _, glitch_types_in_testset_L1_O3b = glitches_in_testset(glitch_gps_times, glitch_duration, glitch_snrs, glitch_types, segments_csv='./O3b/L1_O3b_GPS.csv')

print(glitch_types_in_testset_H1)

H1 O3a contains 80763 glitches


80763it [00:17, 4585.27it/s]


There were 29363 glitches present in the testing data for H1 O3a
H1 O3a contains 127500 glitches


127500it [00:38, 3347.13it/s]


There were 44856 glitches present in the testing data for H1 O3a
H1 O3a contains 99568 glitches


99568it [00:29, 3364.66it/s]


There were 45556 glitches present in the testing data for H1 O3a
H1 O3a contains 192693 glitches


192693it [00:56, 3393.67it/s]

There were 93878 glitches present in the testing data for H1 O3a
[7, 3, 10, 10, 1, 9, 10, 9, 9, 10, 16, 16, 9, 12, 10, 9, 10, 9, 9, 10, 10, 9, 10, 10, 9, 9, 9, 10, 10, 9, 10, 10, 16, 10, 10, 10, 16, 10, 9, 9, 9, 10, 9, 9, 9, 10, 9, 9, 10, 10, 9, 10, 9, 10, 16, 9, 9, 10, 10, 9, 10, 9, 9, 10, 9, 10, 9, 10, 10, 10, 15, 10, 5, 7, 10, 10, 10, 17, 16, 9, 10, 10, 9, 10, 10, 9, 16, 15, 10, 10, 9, 16, 9, 9, 10, 10, 9, 9, 9, 9, 10, 10, 10, 9, 10, 9, 10, 9, 10, 9, 16, 10, 9, 9, 16, 9, 16, 3, 7, 9, 9, 10, 7, 16, 10, 9, 9, 9, 9, 10, 9, 10, 9, 16, 19, 9, 10, 9, 7, 10, 9, 10, 9, 10, 10, 9, 3, 16, 16, 9, 10, 12, 9, 9, 7, 9, 9, 5, 10, 10, 9, 9, 9, 9, 10, 16, 9, 9, 3, 10, 9, 10, 9, 9, 9, 9, 9, 10, 9, 9, 10, 9, 9, 9, 21, 10, 5, 10, 9, 16, 16, 16, 10, 10, 10, 9, 10, 16, 10, 16, 16, 9, 9, 9, 10, 9, 9, 9, 10, 9, 16, 10, 9, 9, 3, 9, 9, 9, 10, 10, 10, 10, 10, 15, 12, 16, 9, 10, 16, 9, 16, 10, 16, 10, 16, 10, 10, 16, 16, 21, 10, 9, 9, 5, 9, 15, 14, 10, 9, 10, 16, 10, 7, 10, 16, 9, 16, 10, 7, 3, 3, 9, 9, 3, 7, 

In [110]:
all_glitches = glitch_types_in_testset_H1_O3a + glitch_types_in_testset_H1_O3b + glitch_types_in_testset_L1_O3a + glitch_types_in_testset_L1_O3b
unique, counts = np.unique(all_glitches, return_counts=True)
dict(zip(unique, counts))

{0: 240,
 1: 414,
 2: 1503,
 3: 18729,
 4: 70,
 5: 11751,
 6: 523,
 7: 14547,
 8: 560,
 9: 20342,
 10: 16678,
 11: 11498,
 12: 19249,
 13: 4966,
 14: 379,
 15: 2718,
 16: 65673,
 17: 214,
 18: 15601,
 19: 1453,
 20: 73,
 21: 6472}

In [116]:
# create test data
team_names = ['Sage_ub', 'Sage', 'PyCBC']
detfiles = ['H1_O3a', 'L1_O3a']
coords = itertools.product(team_names, detfiles)

# figsize - (width, height)
# subplots - (nrows, ncols)
fig = plt.figure(figsize=(10.*3, 7.*4), constrained_layout=True)
gs = fig.add_gridspec(4, 3, width_ratios=[2, 2, 1], height_ratios=[2, 2, 2, 2])

## Glitch type frequency bar plot for training data
ax1 = fig.add_subplot(gs[0, :])
ax1.bar(glitch_alias, counts, width=1.0)
ax1.set_xlabel('Glitch Type', fontsize=22)
ax1.set_ylabel('N Glitches in Training Data', fontsize=22)
ax1.tick_params(labelsize=22)
ax1.set_yscale('log')

## Glitch type acronyms and legend for swarmplot
ax2 = fig.add_subplot(gs[1:, -1])

for n, (galias, gname) in enumerate(zip(glitch_alias, glitch_names)):
    ax2.text(0.05, 0.75-n*0.03, '{} = {}'.format(galias, gname), horizontalalignment='left', fontsize=22)

sage_patch = mpatches.Patch(color='#512888', label='Sage')
pycbc_patch = mpatches.Patch(color='#EB6123', label='PyCBC')
# left/right, up/down, width, height
plt.legend(handles=[sage_patch, pycbc_patch], bbox_to_anchor=(0.15, -2.4, 1., 6.5), loc='center left', frameon=False, fontsize=26, handlelength=2.6)
# Remove borders and ticks
ax2.set_xticks([])
ax2.set_yticks([])
for pos in ['right', 'top', 'bottom', 'left']: 
    plt.gca().spines[pos].set_visible(False) 


## Top glitches comparison
topgs = []
topgs.append(fig.add_subplot(gs[1, 0]))
topgs.append(fig.add_subplot(gs[1, 1]))
topgs.append(fig.add_subplot(gs[2, 0]))
topgs.append(fig.add_subplot(gs[2, 1]))
topgs.append(fig.add_subplot(gs[3, 0]))
topgs.append(fig.add_subplot(gs[3, 1]))


for num, ((tname, detfile), ax) in enumerate(zip(coords, topgs)):
    print('\nCurrent coord: {}, {}'.format(detfile, tname))

    if tname in ['Sage']:
        team_data = team_1
        team_name = tname
        alt_team_data = team_2
        alt_team_name = 'PyCBC'
    elif tname in ['Sage_ub']:
        with open('noise_stats.pickle', 'rb') as handle:
            noise_stats = pickle.load(handle)
        team_data = {'noise_stats': noise_stats}
        team_name = tname
        alt_team_data = team_2
        alt_team_name = 'PyCBC'
    elif tname in ['PyCBC']:
        team_data = team_2
        team_name = tname
        with open('noise_stats.pickle', 'rb') as handle:
            noise_stats = pickle.load(handle)
        alt_team_data = {'noise_stats': noise_stats}
        alt_team_name = 'Sage_ub'
    
    print(team_name, alt_team_name)
    # Convert triggers to FAR/month
    team_stats_in_far = [np.log10(len(team_data['noise_stats'][team_data['noise_stats'] > foo])) for foo in save_glitch_data[team_name][detfile][:,0]]
    team_triggers = np.column_stack((save_glitch_data[team_name][detfile][:,3], team_stats_in_far))
    # alt_team
    alt_team_stats_in_far = [np.log10(len(alt_team_data['noise_stats'][alt_team_data['noise_stats'] > foo])) for foo in save_glitch_data[alt_team_name][detfile][:,0]]
    alt_team_triggers = np.column_stack((save_glitch_data[alt_team_name][detfile][:,3], alt_team_stats_in_far))

    # Iterate through all glitch types
    save_team_triggers = {}
    save_team_triggers_nums = {}
    save_alt_team_triggers = {}
    save_alt_team_triggers_nums = {}
    for i in range(len(glitch_names)):
        # Get all triggers for a particular glitch type
        save_team_triggers[i] = team_triggers[:,1][team_triggers[:,0] == i]
        save_alt_team_triggers[i] = alt_team_triggers[:,1][alt_team_triggers[:,0] == i]
        # glitch_type, trigger_value
        try: 
            save_team_triggers_nums[i] = np.min(team_triggers[:,1][team_triggers[:,0] == i])
        except: 
            pass
        # Alt team
        try: 
            save_alt_team_triggers_nums[i] = np.min(alt_team_triggers[:,1][alt_team_triggers[:,0] == i]) 
        except: 
            pass

    # Which triggers types have the highest trigger values
    highest_team_triggers = dict(sorted(save_team_triggers_nums.items(), key=lambda item: item[1], reverse=False))
    print(highest_team_triggers)
    highest_alt_team_triggers = dict(sorted(save_alt_team_triggers_nums.items(), key=lambda item: item[1], reverse=False))
    top10_glitches_team = list(highest_team_triggers.keys())[:3]
    top10_glitches_alt_team = top10_glitches_team.copy()

    team_data = [save_team_triggers[i][save_team_triggers[i] < 5] for i in top10_glitches_team]
    team_swarm_data = [save_team_triggers[i][save_team_triggers[i] < 4] for i in top10_glitches_team]
    alt_team_data = [save_alt_team_triggers[i][save_alt_team_triggers[i] < 5] for i in top10_glitches_alt_team]
    alt_team_swarm_data = [save_alt_team_triggers[i][save_alt_team_triggers[i] < 4] for i in top10_glitches_alt_team]

    # Make violin plot using searborn
    team_labels = [names_to_alias[map_to_names[foo]] for foo in top10_glitches_team]
    alt_team_labels = [names_to_alias[map_to_names[foo]] for foo in top10_glitches_alt_team]

    # Team data
    team_violin_data = {key:val for key, val in zip(team_labels, team_data)}
    team_swarm_datadict = {key:val for key, val in zip(team_labels, team_swarm_data)}
    # Alt team data
    alt_team_violin_data = {key:val for key, val in zip(alt_team_labels, alt_team_data)}
    alt_team_swarm_datadict = {key:val for key, val in zip(alt_team_labels, alt_team_swarm_data)}

    df = pd.DataFrame([[label, val, team_name] for label, values in team_violin_data.items() for val in values]
                    + [[label, val, alt_team_name] for label, values in alt_team_violin_data.items() for val in values],
                    columns=['Glitch Types ({}, {})'.format(team_name, detfile.replace('_', ' ')), r'$\mathregular{log}_{\mathregular{10}}(\mathregular{FAR/month})$', 'source'])

    swarmdf = pd.DataFrame([[label, val, team_name] for label, values in team_swarm_datadict.items() for val in values]
                    + [[label, val, alt_team_name] for label, values in alt_team_swarm_datadict.items() for val in values],
                    columns=['Glitch Types ({}, {})'.format(team_name, detfile.replace('_', ' ')), r'$\mathregular{log}_{\mathregular{10}}(\mathregular{FAR/month})$', 'source'])

    # ax = sns.violinplot(df, cut=0, orient='h', y='label', x='value', hue='source', inner='quart')
    colors = ['#512888', '#EB6123'] if team_name in ['Sage', 'Sage_ub'] else ['#EB6123', '#512888']
    label_team_name = 'Sage' if team_name in ['Sage', 'Sage_ub'] else 'PyCBC'
    swarm = sns.swarmplot(swarmdf, dodge=False, legend=False, ax=ax, orient='h', y='Glitch Types ({}, {})'.format(team_name, detfile.replace('_', ' ')), 
                  x=r'$\mathregular{log}_{\mathregular{10}}(\mathregular{FAR/month})$', size=10, palette=colors, hue='source')

    # Thesholds in FAR
    colors = np.array([raspberry, dark_red, dark_blue, dark_green, dark_purple])[::-1]
    thresh_names = ['1-per-month', '10-per-month', '100-per-month', '1000-per-month', '10000-per-month']
    thresh_vals = [np.log10(1), np.log10(10), np.log10(100), np.log10(1000), np.log10(10000)]
    thresh = {name:val for name, val in zip(thresh_names, thresh_vals)}
    for (thresh_name, thresh), c in zip(thresh.items(), colors):
        ax.axvline(thresh, ls='--', linewidth=1., label=thresh_name, color='k')
    
    ax.set_xlabel(r'$\mathregular{log}_{\mathregular{10}}(\mathregular{FAR/month})$', fontsize=22)
    ax.set_ylabel('Glitch Types ({}, {})'.format(team_name, detfile.replace('_', ' ')), fontsize=22)
    ax.tick_params(labelsize=22)

    ax = swarm.axes
    ax.invert_xaxis()
    ax.grid(axis='x')

plt.savefig('./evaluation_plots/glitch_swarmplot_pycbc_sage.png')
plt.show()


Current coord: H1_O3a, Sage_ub
Sage_ub PyCBC
{19: 2.1522883443830563, 20: 2.2430380486862944, 16: 2.2695129442179165, 21: 2.385606273598312, 10: 2.597695185925512, 7: 2.632457292184724, 3: 2.6541765418779604, 15: 2.8444771757456815, 17: 2.8444771757456815, 9: 2.8615344108590377, 11: 2.991226075692495}

Current coord: L1_O3a, Sage_ub
Sage_ub PyCBC
{7: 1.6434526764861874, 10: 2.2695129442179165, 16: 2.3242824552976926, 9: 2.437750562820388, 18: 2.4653828514484184, 13: 2.7701152947871015}

Current coord: H1_O3a, Sage
Sage PyCBC
{21: 0.8450980400142568, 20: 1.6020599913279625, 3: 1.863322860120456, 11: 2.2174839442139063, 16: 2.285557309007774, 15: 2.3636119798921444, 12: 2.680335513414563, 7: 2.7965743332104296, 10: 2.808885867359812, 9: 2.8779469516291885, 19: 3.1931245983544616, 6: 3.5435714239623652, 0: 3.619719265611727, 1: 3.757547853469244, 5: 3.762378429311964, 17: 3.8490506905695123, 14: 3.900476371389257, 18: 4.101231386790699, 8: 4.28616444665352, 4: 4.636136708051051, 2: 4.652

### Robustness to PSD realisations

In [ ]:
import pycbc
from pycbc import DYN_RANGE_FAC
from pycbc.filter import highpass as pycbc_highpass
from pycbc.psd import inverse_spectrum_truncation, welch, interpolate
from pycbc.types import TimeSeries, FrequencySeries
from pycbc.types import complex_same_precision_as

from noise_generator import NoiseGenerator
from make_signal import generate

In [ ]:
h1_psds_hdf="./tmp/psds_H1_latter51days_20s.hdf"
l1_psds_hdf="./tmp/psds_L1_latter51days_20s.hdf"

h1_psds = h5py.File(h1_psds_hdf, 'r')
shape_h1_psds = dict(h1_psds.attrs)['shape']
l1_psds = h5py.File(l1_psds_hdf, 'r')
shape_l1_psds = dict(h1_psds.attrs)['shape']

In [ ]:
fs = 2048. #Hz
sample_length_in_s = 20.0
noise_low_freq_cutoff = 15.0
signal_low_freq_cutoff = 20.0
trunc_method = 'hann'
whiten_padding = 5.0 # seconds

In [ ]:
def inv_spec_trunc(psd, max_filter_len):
    # Interpolate and smooth to the desired corruption length
    psd = inverse_spectrum_truncation(psd,
                                    max_filter_len=max_filter_len,
                                    low_frequency_cutoff=20.0,
                                    trunc_method='hann')
    return psd

In [ ]:
noise_generator = NoiseGenerator(delta_f=1./sample_length_in_s,
                                 sample_rate=2048.,
                                 low_frequency_cutoff=15.0)

In [ ]:
def psd_to_asd(psd, start_time, end_time,
               sample_rate=2048.,
               low_frequency_cutoff=15.0,
               filter_duration=128):
    
    psd = psd.copy()

    flen = int(sample_rate / psd.delta_f) // 2 + 1
    oldlen = len(psd)
    psd.resize(flen)

    # Want to avoid zeroes in PSD.
    max_val = psd.max()
    for i in range(len(psd)):
        if i >= (oldlen-1):
            psd.data[i] = psd[oldlen - 2]
        if psd[i] == 0:
            psd.data[i] = max_val

    fil_len = int(filter_duration * sample_rate)
    wn_dur = int(end_time - start_time) + 2 * filter_duration
    if psd.delta_f >= 1. / (2.*filter_duration):
        # If the PSD is short enough, this method is less memory intensive than
        # resizing and then calling inverse_spectrum_truncation
        psd = pycbc.psd.interpolate(psd, 1.0 / (2. * filter_duration))
        # inverse_spectrum_truncation truncates the inverted PSD. To truncate
        # the non-inverted PSD we give it the inverted PSD to truncate and then
        # invert the output.
        psd = 1. / pycbc.psd.inverse_spectrum_truncation(
                                1./psd,
                                fil_len,
                                low_frequency_cutoff=low_frequency_cutoff,
                                trunc_method='hann')
        psd = psd.astype(complex_same_precision_as(psd))
        # Zero-pad the time-domain PSD to desired length. Zeroes must be added
        # in the middle, so some rolling between a resize is used.
        psd = psd.to_timeseries()
        psd.roll(fil_len)
        psd.resize(int(wn_dur * sample_rate))
        psd.roll(-fil_len)
        # As time series is still mirrored the complex frequency components are
        # 0. But convert to real by using abs as in inverse_spectrum_truncate
        psd = psd.to_frequencyseries()

    kmin = int(low_frequency_cutoff / psd.delta_f)
    psd[:kmin].clear()
    asd = (psd.squared_norm())**0.25
    return asd

In [ ]:
det = 'H1'

In [ ]:
# Making a single signal to be placed in different PSD realisations
waveform_kwargs = {}
waveform_kwargs['delta_t'] = 1./2048.
waveform_kwargs['f_lower'] = 20.0 # Hz
waveform_kwargs['approximant'] = 'IMRPhenomXPHM'
waveform_kwargs['f_ref'] = 20.0 # Hz
waveform_kwargs['mass1'] = 20.0
waveform_kwargs['mass2'] = 10.0
waveform_kwargs['spin1z'] = 0.0
waveform_kwargs['spin2z'] = 0.0
waveform_kwargs['inclination'] = 0.0
waveform_kwargs['coa_phase'] = 2.45
waveform_kwargs['injection_time'] = 1244166018
waveform_kwargs['polarization'] = 2.34
waveform_kwargs['dec'] = 0.65
waveform_kwargs['ra'] = 4.67
waveform_kwargs['tc'] = 13.2
waveform_kwargs['distance'] = 3000.0 # Mpc

signal = generate(waveform_kwargs)
crop = int(whiten_padding/2.*2048.)
signal = signal[0][crop:-crop]

In [ ]:
plt.plot(signal)

In [ ]:
if det == 'H1':
    total_num = int(shape_h1_psds[0])
    det_psds = h1_psds['data']
elif det == 'L1':
    total_num = int(shape_l1_psds[0])
    det_psds = l1_psds['data']

for cidx in range(total_num):
    psd = det_psds[cidx]
    psd = FrequencySeries(psd, delta_f=1./sample_length_in_s)
    psd = interpolate(psd, 1./sample_length_in_s)
    max_filter_len = int(round(0.1 * fs))
    psd = inv_spec_trunc(psd, max_filter_len)
    asd = psd_to_asd(psd, 0.0, 20.0,
                    sample_rate=2048.,
                    low_frequency_cutoff=15.0,
                    filter_duration=20.0)
    # Create noise realisation with given PSD
    noise = noise_generator(2.5, 2.5+sample_length_in_s, seed=cidx, asd=asd)
    noise = noise.numpy()
    # Cropping
    crop = slice(int(whiten_padding/2.*fs), -int(whiten_padding/2.*fs))
    noise = noise[crop]
    # Create noisy signal sample
    noisy_signal = signal + noise
    plt.plot(noisy_signal)
    raise

In [ ]:
noise = np.random.normal(0, scale=1, size=10_000)
signal = 1.0*np.sin(2.*np.pi*2*np.linspace(0, 10, 10_000) + 0.0) + 0.0
noisy_sample = signal + noise
plt.plot(noisy_sample)
plt.plot(signal)